In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 11.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import optuna
import pickle

from sklearn.model_selection import (GridSearchCV, StratifiedKFold, KFold, cross_val_score)
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report,
                             roc_curve, roc_auc_score, precision_recall_curve, matthews_corrcoef)

In [ ]:
data = pd.read_csv('/content/training_data.csv')
data = data.drop('sequence', axis = 1)
data.head()

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeavyAtoms,NumAtoms,NumHeteroatoms,NumAmideBonds,FractionCSP3,NumRings,NumAromaticRings,NumAliphaticRings,NumSaturatedRings,NumHeterocycles,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumSpiroAtoms,NumBridgeheadAtoms,NumAtomStereoCenters,NumUnspecifiedAtomStereoCenters,labuteASA,tpsa,CrippenClogP,CrippenMR,chi0v,chi1v,chi2v,chi3v,chi4v,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi,cofactor,target
0,1.0,2.0,1.0,3.0,4.0,2.0,2.0,1.0,1.0,4.0,0.0,3.0,2.0,1.0,5.0,7.0,0.329248,0.294038,0.341565,0.101232,0.132560,0.343678,-0.264682,0.449203,-0.130112,0.238872,-0.467052,-0.224363,0.352041,-0.344090,0.140551,-0.578147,-0.323673,0.684536,-0.598860,0.384600,0.122938,0.615171,0.860627,0.275864,0.191519,-0.156134,0.272819,-0.150450,-0.067301,0.668748,-0.454932,-0.240006,0.262655,0.021360,0.255669,-0.293530,0.314213,-0.446205,0.546025,-0.393203,-0.387536,-0.083074,-0.192116,8,1
1,1.0,4.0,2.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,0.0,2.0,6.0,0.0,1.0,7.0,0.329247,0.294231,0.341586,0.101389,0.131927,0.343852,-0.265001,0.449447,-0.130146,0.239260,-0.466380,-0.224489,0.352128,-0.343958,0.139796,-0.578037,-0.323229,0.684312,-0.598951,0.384706,0.122993,0.614787,0.860599,0.275912,0.191595,-0.156446,0.272693,-0.150249,-0.067265,0.668224,-0.454817,-0.240145,0.262004,0.021147,0.255222,-0.293445,0.314236,-0.446417,0.546032,-0.393267,-0.387402,-0.083250,-0.192060,8,1
2,5.0,2.0,9.0,1.0,3.0,5.0,5.0,2.0,6.0,7.0,8.0,6.0,3.0,2.0,4.0,4.0,0.614713,0.420567,0.673089,0.208772,0.242736,0.498475,-0.544441,0.785882,-0.259640,0.441833,-0.887025,-0.313468,0.657169,-0.524298,0.314166,-0.924216,-0.572622,1.160306,-1.026042,0.566252,0.240411,1.181413,1.387715,0.451344,0.259354,-0.314586,0.520666,-0.245381,-0.106600,1.214760,-0.692977,-0.360328,0.504484,-0.010680,0.459596,-0.595511,0.494463,-0.788223,0.836717,-0.683155,-0.705708,-0.242489,-0.416548,18,1
3,8.0,3.0,5.0,3.0,5.0,5.0,5.0,4.0,4.0,6.0,5.0,4.0,3.0,4.0,4.0,5.0,0.626463,0.425043,0.687694,0.212831,0.243548,0.505288,-0.559207,0.798803,-0.265476,0.452208,-0.907827,-0.316099,0.675179,-0.532286,0.320083,-0.937916,-0.581937,1.179896,-1.042855,0.574637,0.245350,1.207134,1.411677,0.458814,0.260606,-0.322757,0.531085,-0.250826,-0.106850,1.237135,-0.701268,-0.365639,0.512527,-0.011627,0.469187,-0.609093,0.501323,-0.803722,0.849435,-0.695537,-0.721361,-0.251902,-0.424925,18,1
4,4.0,4.0,7.0,0.0,4.0,4.0,5.0,3.0,3.0,5.0,10.0,9.0,4.0,2.0,5.0,3.0,0.614850,0.420844,0.672392,0.207234,0.243664,0.499066,-0.545793,0.784143,-0.261379,0.440877,-0.887794,-0.311650,0.657948,-0.525584,0.316758,-0.923787,-0.571821,1.161412,-1.026236,0.565786,0.240541,1.181869,1.389740,0.452649,0.260228,-0.314642,0.521174,-0.246820,-0.107791,1.214476,-0.693815,-0.360968,0.504868,-0.011163,0.459429,-0.596054,0.493541,-0.787897,0.836684,-0.682053,-0.704940,-0.240326,-0.417047,18,1


In [ ]:
data['target'].value_counts()

,count
target,
0,1000
1,154


In [ ]:
X = data.drop('target', axis = 1)
X = X.iloc[:,:-1]
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_test.value_counts()

,count
target,
0,200
1,31


LGBM

---



In [ ]:
def objective(trial):
    # Подбор гиперпараметров
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'num_leaves': trial.suggest_int('num_leaves', 5, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'random_state': 42,
        'n_jobs': -1
    }


    model = LGBMClassifier(**param, verbose = -1, is_unbalance=True)


    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='matthews_corrcoef')


    return scores.mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print("Лучшие гиперпараметры:")
print(study.best_params)

[I 2025-06-19 10:47:28,210] A new study created in memory with name: no-name-e2b753b2-9dd0-414a-bc80-a3e66f0fc4aa
[I 2025-06-19 10:47:29,891] Trial 0 finished with value: 0.29177272852705044 and parameters: {'n_estimators': 98, 'learning_rate': 0.0989984067755145, 'max_depth': 11, 'num_leaves': 24, 'min_child_samples': 13, 'subsample': 0.5202865598033232, 'colsample_bytree': 0.5724011536783833, 'reg_alpha': 4.1047685123082306e-05, 'reg_lambda': 0.0003018863443217155, 'class_weight': None, 'boosting_type': 'dart'}. Best is trial 0 with value: 0.29177272852705044.
[I 2025-06-19 10:47:30,869] Trial 1 finished with value: 0.15600711665318068 and parameters: {'n_estimators': 113, 'learning_rate': 0.02675428110971017, 'max_depth': 19, 'num_leaves': 47, 'min_child_samples': 50, 'subsample': 0.7979808651266012, 'colsample_bytree': 0.8839572951092755, 'reg_alpha': 0.009370361241777543, 'reg_lambda': 5.616446246276178e-06, 'class_weight': 'balanced', 'boosting_type': 'gbdt'}. Best is trial 0 wit

Лучшие гиперпараметры:
{'n_estimators': 105, 'learning_rate': 0.3329029817527149, 'max_depth': 24, 'num_leaves': 39, 'min_child_samples': 31, 'subsample': 0.650447871963091, 'colsample_bytree': 0.6572857231299954, 'reg_alpha': 0.3833274857496429, 'reg_lambda': 4.2843636071621694e-05, 'class_weight': None, 'boosting_type': 'dart'}


In [ ]:
LGBM_model = LGBMClassifier(**study.best_params, verbose = -1, is_unbalance=True, random_state = 42)
LGBM_model.fit(X_train, y_train)

# Оценка на тестовой выборке
y_pred = LGBM_model.predict(X_test)
print("MCC на тесте:", matthews_corrcoef(y_test, y_pred))

MCC на тесте: 0.41459442973240973


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(LGBM_model, X, y, cv=skf, scoring='matthews_corrcoef')
for i in range(5):
  print(f'Fold {i+1}: MCC = {scores[i]}')
print('Mean MCC:', scores.mean())

Fold 1: MCC = 0.5024413136361183
Fold 2: MCC = 0.3443818816629602
Fold 3: MCC = 0.43517012163468355
Fold 4: MCC = 0.5561989958992111
Fold 5: MCC = 0.460360868421186
Mean MCC: 0.45971063625083186


In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[185  15]
 [ 16  15]]


XGBoost

---



In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 50),  # для несбалансированных классов
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42,
        'n_jobs': -1
    }

    model = XGBClassifier(**param)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='matthews_corrcoef')

    return scores.mean()

In [ ]:
study_2 = optuna.create_study(direction='maximize')
study_2.optimize(objective, n_trials=100)
print("Лучшие гиперпараметры:")
print(study_2.best_params)

[I 2025-06-19 10:49:37,176] A new study created in memory with name: no-name-dfaab49a-5c68-488c-ad9b-7e7b0aa18a3f
[I 2025-06-19 10:49:40,420] Trial 0 finished with value: 0.233868452325652 and parameters: {'n_estimators': 149, 'learning_rate': 0.014722497834073082, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.8091332737975302, 'colsample_bytree': 0.5007173227712607, 'gamma': 3.275562746197615, 'reg_alpha': 0.06154292095749925, 'reg_lambda': 1.8979995104073252e-07, 'scale_pos_weight': 15.642220305484907}. Best is trial 0 with value: 0.233868452325652.
[I 2025-06-19 10:49:46,306] Trial 1 finished with value: 0.31574249213209227 and parameters: {'n_estimators': 88, 'learning_rate': 0.018782790669666465, 'max_depth': 15, 'min_child_weight': 2, 'subsample': 0.7092279789215277, 'colsample_bytree': 0.7324182033250353, 'gamma': 2.378293584914905, 'reg_alpha': 4.511571618601955e-06, 'reg_lambda': 0.0006876549362914376, 'scale_pos_weight': 6.7448393984066435}. Best is trial 1 with val

Лучшие гиперпараметры:
{'n_estimators': 64, 'learning_rate': 0.040583824783414914, 'max_depth': 24, 'min_child_weight': 5, 'subsample': 0.63058960076661, 'colsample_bytree': 0.8872861469296004, 'gamma': 0.8323214392047904, 'reg_alpha': 1.754542359323983e-05, 'reg_lambda': 0.007272675511428947, 'scale_pos_weight': 5.509709563625485}


In [ ]:
XGB_model = XGBClassifier(**study_2.best_params, random_state = 42)
XGB_model.fit(X_train, y_train)

# Оценка на тестовой выборке
y_pred = XGB_model.predict(X_test)
print("MCC на тесте:", matthews_corrcoef(y_test, y_pred))

MCC на тесте: 0.44980282418539935


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(XGB_model, X, y, cv=skf, scoring='matthews_corrcoef')
for i in range(5):
  print(f'Fold {i+1}: MCC = {scores[i]}')
print('Mean MCC:', scores.mean())

Fold 1: MCC = 0.46289385495117447
Fold 2: MCC = 0.3049116880692218
Fold 3: MCC = 0.39144068062940635
Fold 4: MCC = 0.5169304291561246
Fold 5: MCC = 0.3624711931363078
Mean MCC: 0.4077295691884471


In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[192   8]
 [ 18  13]]


Random Forest

---



In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced', 'balanced_subsample']),
        'random_state': 42,
        'n_jobs': -1
    }

    model = RandomForestClassifier(**param)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='matthews_corrcoef')

    return scores.mean()

In [ ]:
study_3 = optuna.create_study(direction='maximize')
study_3.optimize(objective, n_trials=250)

print("Лучшие гиперпараметры:")
print(study_3.best_params)

[I 2025-06-19 11:43:15,799] A new study created in memory with name: no-name-6a7664b7-63b7-436e-aa6d-ab9b84b5eca0
[I 2025-06-19 11:43:21,742] Trial 0 finished with value: 0.13940687374651808 and parameters: {'n_estimators': 54, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_features': None, 'bootstrap': False, 'class_weight': None}. Best is trial 0 with value: 0.13940687374651808.
[I 2025-06-19 11:43:37,296] Trial 1 finished with value: 0.12560401708912997 and parameters: {'n_estimators': 117, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None, 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.13940687374651808.
[I 2025-06-19 11:43:43,095] Trial 2 finished with value: 0.2723133122471336 and parameters: {'n_estimators': 182, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 2 with value: 0.272313312247133

Лучшие гиперпараметры:
{'n_estimators': 115, 'max_depth': 29, 'min_samples_split': 11, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False, 'class_weight': 'balanced'}


In [ ]:
RF_model

RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=29,
                       min_samples_leaf=2, min_samples_split=11,
                       n_estimators=115, random_state=42)

In [ ]:
#RF_model = RandomForestClassifier(**study_3.best_params, random_state = 42)
RF_model = RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=29,
                       min_samples_leaf=2, min_samples_split=11,
                       n_estimators=115, random_state=42)
RF_model.fit(X_train, y_train)

# Оценка на тестовой выборке
y_pred = RF_model.predict(X_test)
print("MCC на тесте:", matthews_corrcoef(y_test, y_pred))

MCC на тесте: 0.5147297199383514


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(RF_model, X, y, cv=skf, scoring='matthews_corrcoef')
for i in range(5):
  print(f'Fold {i+1}: MCC = {scores[i]}')
print('Mean MCC:', scores.mean())

Fold 1: MCC = 0.3790322580645161
Fold 2: MCC = 0.2981574687262119
Fold 3: MCC = 0.335648539012593
Fold 4: MCC = 0.4486898490551812
Fold 5: MCC = 0.33336640047624816
Mean MCC: 0.3589789030669501


In [ ]:
models = [LGBM_model, XGB_model, RF_model]
models_names = ['LGBM', 'XGBoost', 'Random_Forest']

def get_metrics(models_list, models_names):
  roc_auc_metrics = []
  f1_score_metrics = []
  precision_metrics = []
  recall_metrics = []
  mcc_metrics = []
  FP = []
  FN = []
  TP = []
  TN = []


  for model in models_list:
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    #threshold = 0.8
    #y_pred = (y_proba > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    roc_auc_metrics.append(roc_auc_score(y_test, y_proba))
    f1_score_metrics.append(f1_score(y_test, y_pred))
    precision_metrics.append(precision_score(y_test, y_pred))
    recall_metrics.append(recall_score(y_test, y_pred))
    mcc_metrics.append(matthews_corrcoef(y_test, y_pred))
    FP.append(fp)
    FN.append(fn)
    TN.append(tn)
    TP.append(tp)

  data = pd.DataFrame({'model' : models_names,
                       'roc/auc': roc_auc_metrics,
                       'f1': f1_score_metrics,
                       'precision': precision_metrics,
                       'recall': recall_metrics,
                       'mcc': mcc_metrics,
                       'TN': TN,
                       'FP': FP,
                       'FN': FN,
                       'TP': TP})
  return data

In [ ]:
data = get_metrics(models, models_names)

In [ ]:
data

,model,roc/auc,f1,precision,recall,mcc,TN,FP,FN,TP
0,LGBM,0.773065,0.491803,0.500000,0.483871,0.414594,185,15,16,15
1,XGBoost,0.789355,0.500000,0.619048,0.419355,0.449803,192,8,18,13
2,Random_Forest,0.787581,0.521739,0.800000,0.387097,0.514730,197,3,19,12


In [ ]:
import joblib

joblib.dump(RF_model, 'seq_model.pkl')

['seq_model.pkl']